In [70]:
import mxnet as mx
import cv2
import numpy as np

In [71]:
def get_image(filename):
    img = cv2.imread(filename)  # read image in b,g,r order
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   # change to r,g,b order
    img = cv2.resize(img, (224, 224))  # resize to 224*224 to fit model
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)  # change to (channel, height, width)
    img = img[np.newaxis, :]  # extend to (example, channel, heigth, width)
    return img

In [72]:
def get_iteror(batchsize, datashape = (3,224,224)):
    train = mx.io.ImageRecordIter(
    #    batch_size = batchsize,
        path_imgrec = 'E:/train/model_train.rec',
        data_shape = datashape,
        shuffle = True,
        rand_crop = True,
        rand_mirror = True,
        data_name = 'data_name',
        lable_name = 'softmax_label'
    )
    
    test = mx.io.ImageRecordIter(
        batch_size = batchsize,
        path_imgrec = 'E:/train/model_test.rec',
        data_shape = datashape,
        rand_crop = False,
        rand_mirror = False,
        data_name = 'data_name',
        lable_name = 'softmax_label'
    )
    return(train, test)

In [73]:
def model_extract(perfix, epoch):
    sym, arg_params, aux_params = mx.model.load_checkpoint(perfix, epoch)
    internels = sym.get_internals()
    fea_symbol = internels["fc1_output"]
    return(fea_symbol, arg_params, aux_params)

In [74]:
fea_model, arg_params, aux_params = model_extract('D:/gitcode/DataScience/Python/python-mxnet-notebook/how_to/resnet-50',0)

In [75]:
sym, arg_params, aux_params = mx.model.load_checkpoint('D:/gitcode/DataScience/Python/python-mxnet-notebook/how_to/resnet-50',0)

In [76]:
internals = sym.get_internals()
internals.list_outputs()

['data',
 'bn_data_gamma',
 'bn_data_beta',
 'bn_data_output',
 'conv0_weight',
 'conv0_output',
 'bn0_gamma',
 'bn0_beta',
 'bn0_output',
 'relu0_output',
 'pooling0_output',
 'stage1_unit1_bn1_gamma',
 'stage1_unit1_bn1_beta',
 'stage1_unit1_bn1_output',
 'stage1_unit1_relu1_output',
 'stage1_unit1_conv1_weight',
 'stage1_unit1_conv1_output',
 'stage1_unit1_bn2_gamma',
 'stage1_unit1_bn2_beta',
 'stage1_unit1_bn2_output',
 'stage1_unit1_relu2_output',
 'stage1_unit1_conv2_weight',
 'stage1_unit1_conv2_output',
 'stage1_unit1_bn3_gamma',
 'stage1_unit1_bn3_beta',
 'stage1_unit1_bn3_output',
 'stage1_unit1_relu3_output',
 'stage1_unit1_conv3_weight',
 'stage1_unit1_conv3_output',
 'stage1_unit1_sc_weight',
 'stage1_unit1_sc_output',
 '_plus0_output',
 'stage1_unit2_bn1_gamma',
 'stage1_unit2_bn1_beta',
 'stage1_unit2_bn1_output',
 'stage1_unit2_relu1_output',
 'stage1_unit2_conv1_weight',
 'stage1_unit2_conv1_output',
 'stage1_unit2_bn2_gamma',
 'stage1_unit2_bn2_beta',
 'stage1_unit2_

In [77]:
aa = sym.get_internals()
aa.list_outputs()

['data',
 'bn_data_gamma',
 'bn_data_beta',
 'bn_data_output',
 'conv0_weight',
 'conv0_output',
 'bn0_gamma',
 'bn0_beta',
 'bn0_output',
 'relu0_output',
 'pooling0_output',
 'stage1_unit1_bn1_gamma',
 'stage1_unit1_bn1_beta',
 'stage1_unit1_bn1_output',
 'stage1_unit1_relu1_output',
 'stage1_unit1_conv1_weight',
 'stage1_unit1_conv1_output',
 'stage1_unit1_bn2_gamma',
 'stage1_unit1_bn2_beta',
 'stage1_unit1_bn2_output',
 'stage1_unit1_relu2_output',
 'stage1_unit1_conv2_weight',
 'stage1_unit1_conv2_output',
 'stage1_unit1_bn3_gamma',
 'stage1_unit1_bn3_beta',
 'stage1_unit1_bn3_output',
 'stage1_unit1_relu3_output',
 'stage1_unit1_conv3_weight',
 'stage1_unit1_conv3_output',
 'stage1_unit1_sc_weight',
 'stage1_unit1_sc_output',
 '_plus0_output',
 'stage1_unit2_bn1_gamma',
 'stage1_unit2_bn1_beta',
 'stage1_unit2_bn1_output',
 'stage1_unit2_relu1_output',
 'stage1_unit2_conv1_weight',
 'stage1_unit2_conv1_output',
 'stage1_unit2_bn2_gamma',
 'stage1_unit2_bn2_beta',
 'stage1_unit2_

In [78]:
# reset model
mod = mx.mod.Module(symbol=fea_model, context=mx.cpu())

In [79]:
mod.bind(for_training = False,
         data_shapes=[('data', (1,3,224,224))])
mod.set_params(arg_params, aux_params)

In [80]:
img = get_image('E:/train/ALB/img_00003.jpg')

In [81]:
img2 = cv2.cvtColor(cv2.imread('E:/train/ALB/img_00003.jpg'), cv2.COLOR_BGR2RGB)
img2[1]

array([[255, 255, 255],
       [253, 255, 254],
       [254, 255, 255],
       ..., 
       [ 99,  94,  90],
       [101,  96,  92],
       [100,  95,  91]], dtype=uint8)

In [82]:
from collections import namedtuple
Batch = namedtuple('Batch', ['data'])
mod.forward(Batch([mx.nd.array(img)]))

In [83]:
transform_features = mod.get_outputs()[0].asnumpy()

In [84]:
transform_features.shape

(1L, 1000L)

In [85]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
img3 = transform_features[::]
img3.shape
# plt.imshow(img3)

(1L, 1000L)

In [86]:
img2.shape

(720L, 1280L, 3L)